In [1]:
# STANDARD LIBRARY IMPORTS
import re
import csv
import os
import datetime

# PROJECT IMPORTS
from setup import reddit
from settings import *

# THIRD PARTY IMPORTS
import praw

Loaded 0 users who have opted out from follow-ups.
Loaded 1 users who have opted out from reminders.


In [2]:
REMINDER_NUM_COMMENTS_THRESHOLD = 4

In [3]:
subreddit = reddit.subreddit(TARGET_SUBREDDIT)

In [4]:
# overiding this for testing
REMINDER_LOWER_BOUND, REMINDER_UPPER_BOUND = 24,11*24
assert REMINDER_LOWER_BOUND <= REMINDER_UPPER_BOUND

In [5]:
def get_last_processed_post():
    if os.path.exists(LAST_PROCESSED_POST_PATH):
        with open(LAST_PROCESSED_POST_PATH, 'r') as f:
            return f.read().strip()
    else: return None

def set_last_processed_post(post_id):
    with open(LAST_PROCESSED_POST_PATH, 'w') as f:
        f.write(post_id)
    return

In [10]:
get_last_processed_post()

In [14]:
last_processed_post = get_last_processed_post()
new_last_processed_post = None

now = datetime.datetime.utcnow() # using utcnow to align with reddit timezone

for submission in subreddit.new(limit=None):
    submission_age = now - datetime.datetime.fromtimestamp( submission.created_utc )
    submission_age = submission_age.total_seconds() / 60**2 # convert to total hours
    
    print(f'considering post {submission.id}... ', end='')
    
    if submission_age < REMINDER_LOWER_BOUND:
        print(f'skipping young post {submission.title}')
        continue # ignore young posts
        
    if (last_processed_post is not None) and (submission.id == last_processed_post):
        print(f'stopping at last processed post {submission.id}')
        break
    
    if submission_age > REMINDER_UPPER_BOUND:
        print(f'stopping at older post {submission.title}')
        break
        
    print(f'processing post {submission.title}')
    if int(submission.id, 36) > int(new_last_processed_post, 36):
        # keep only the youngest post
        new_last_processed_post = submission.id
        
if new_last_processed_post is not None:
    set_last_processed_post(new_last_processed_post)

considering post t92vd0... skipping young post Mount Baker from Yellow Aster Butte
considering post t5evz5... stopping at last processed post t5evz5


In [6]:
submission.flair.choices()

[{'flair_css_class': '',
  'flair_template_id': '7eeec638-8f07-11ec-8294-2a47d40d3100',
  'flair_text_editable': True,
  'flair_position': 'right',
  'flair_text': 'HPs from op'}]

In [7]:
submission.flair

In [8]:
submission.link_flair_template_id == FLAIR_TEMPLATE_ID

True

In [17]:
def send_reminder(submission):
    '''
    Send a reminder to the author of the given submission, unless either of the following conditions are met:
    - the author has opted out from reminders, or
    - the submission already has a flair indicating it has recieved HPs from OP
    '''
    if (submission.link_flair_template_id == FLAIR_TEMPLATE_ID) or (submission.author.name in REMINDER_OPTED_OUT_USERS):
        # already had points given by OP or opted out from reminders, skip
        return

    if len( submission.comments.list() ) < REMINDER_NUM_COMMENTS_THRESHOLD:
        # not enough comments to message
        return
    
    reminder_subject = f'Did you get good feedback on your photo in /r/{TARGET_SUBREDDIT}?'
    reminder_body = []
    reminder_body.append(f'You recently posted [a photo for critique](https://www.reddit.com{submission.permalink}) in /r/{TARGET_SUBREDDIT}.')
    reminder_body.append('\n')
    reminder_body.append('')
    reminder_body.append('I hope you received some useful feedback!')
    reminder_body.append(f"If you did, you can reply to any comment with '{FEEDBACK_TOKEN.pattern}' to give a Critique Point to the person who provided you with a high quality critique.")
    reminder_body.append('Giving Critique Points is easy, and helps improve the whole community.')
    reminder_body.append(f'See [here]({BOT_INFO_URL}) for more information on Critique Points.')
    reminder_body.append('\n')
    reminder_body.append(f"This message was sent by a bot. To opt out of future reminders such as this one, reply to this PM with '{REMINDER_OPTOUT_PATTERN.pattern}'.")
    
    submission.author.message(reminder_subject, '\n'.join(reminder_body))
    return